In [1]:
from google.colab import drive
drive.mount('content')
%ls
%cd content/MyDrive/RecSys_Data

Mounted at content
content/  sample_data/
/content/content/MyDrive/RecSys_Data


In [2]:
%ls
%cd content/MyDrive/RecSys_Data

articles.csv                        data_generation.ipynb     text_inputs.pt
balanced_political_news_40k_v2.pkl  even_filter_base.csv      text_mask.pt
batched_article_embeddings/         testing_mask.csv          training_mask.csv
BERT/                               test_text_embeddings.pt   validation_mask.csv
BertEmbeddings.ipynb                test_title_embeddings.pt
[Errno 2] No such file or directory: 'content/MyDrive/RecSys_Data'
/content/content/MyDrive/RecSys_Data


In [3]:
%ls

articles.csv                        data_generation.ipynb     text_inputs.pt
balanced_political_news_40k_v2.pkl  even_filter_base.csv      text_mask.pt
batched_article_embeddings/         testing_mask.csv          training_mask.csv
BERT/                               test_text_embeddings.pt   validation_mask.csv
BertEmbeddings.ipynb                test_title_embeddings.pt


In [4]:
import csv
import pickle

In [5]:
### data inspection

In [6]:
articles = []
with (open("balanced_political_news_40k_v2.pkl", "rb")) as openfile:
    while True:
        try:
            articles.append(pickle.load(openfile))
        except EOFError:
            break

In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame(articles);

In [9]:
df.head()

,1741577,2141595,6556505,2867824,5703814,1267386,5140948,6610906,3131521,5276035,...,5260607,2685025,534726,4564071,1273620,1134021,5919097,3443358,5932197,4092559
0,"{'article_id': 1741577, 'url': 'http://www.msn...","{'article_id': 2141595, 'url': 'https://www.vo...","{'article_id': 6556505, 'url': 'https://www.mo...","{'article_id': 2867824, 'url': 'https://slate....","{'article_id': 5703814, 'url': 'https://theint...","{'article_id': 1267386, 'url': 'https://www.al...","{'article_id': 5140948, 'url': 'https://www.vo...","{'article_id': 6610906, 'url': 'https://www.al...","{'article_id': 3131521, 'url': 'https://www.th...","{'article_id': 5276035, 'url': 'https://www.bu...",...,"{'article_id': 5260607, 'url': 'https://thefed...","{'article_id': 2685025, 'url': 'https://www.na...","{'article_id': 534726, 'url': 'https://dailyca...","{'article_id': 4564071, 'url': 'https://www.da...","{'article_id': 1273620, 'url': 'https://www.da...","{'article_id': 1134021, 'url': 'https://dailyc...","{'article_id': 5919097, 'url': 'https://www.na...","{'article_id': 3443358, 'url': 'https://www.br...","{'article_id': 5932197, 'url': 'https://www.da...","{'article_id': 4092559, 'url': 'https://www.br..."


In [10]:
sample = df[1741577].iloc[0]

In [11]:
%pip install print_schema

In [12]:
from print_schema import print_schema

In [13]:
print_schema (sample)

|- article_id	 - <class 'int'>
|- url	 - <class 'str'>
|- title	 - <class 'str'>
|- text	 - <class 'str'>
|- source	 - <class 'str'>
|- source_partisan_score	 - <class 'float'>
|- tweet_id	 - <class 'int'>
|- tweet_screen_name	 - <class 'str'>
|- tweet_created_at	 - <class 'str'>
|- tweet_text	 - <class 'str'>
|- kws_label	 - list [2] <class 'str'>
|- cls_label	 - list [1] <class 'str'>
|- feature_vector	 - list [3000] <class 'float'>
|- topical_vector	 - list [70] <class 'int'>


In [14]:
print(sample['kws_label'])

#cls_label actually matches topical_vector
print(sample['cls_label'])

['health care', 'us 2020 election']
['health care']


In [15]:
##feature vector - this was the feature for classification of topics - very sparse, in reality we may want a BERT model to generate embeddings
##topical vector - for user choice simulation

print(sample['topical_vector'])
print(sample['source_partisan_score'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-2.0


In [16]:
print(sample['url'])

http://www.msnbc.com/rachel-maddow-show/gop-con-man-newly-elected-governor


In [17]:
#what info do we need for our BERT + adversarial autoencoder training?
#article_id, title, text, topical_vector

In [18]:
#what we need for BERT
#article_id, title, text

temp = df.copy()

#what we need for adversarial autoencoder
#article_id, bert_embedding, source_partisan_score

In [19]:
temp.head()

,1741577,2141595,6556505,2867824,5703814,1267386,5140948,6610906,3131521,5276035,...,5260607,2685025,534726,4564071,1273620,1134021,5919097,3443358,5932197,4092559
0,"{'article_id': 1741577, 'url': 'http://www.msn...","{'article_id': 2141595, 'url': 'https://www.vo...","{'article_id': 6556505, 'url': 'https://www.mo...","{'article_id': 2867824, 'url': 'https://slate....","{'article_id': 5703814, 'url': 'https://theint...","{'article_id': 1267386, 'url': 'https://www.al...","{'article_id': 5140948, 'url': 'https://www.vo...","{'article_id': 6610906, 'url': 'https://www.al...","{'article_id': 3131521, 'url': 'https://www.th...","{'article_id': 5276035, 'url': 'https://www.bu...",...,"{'article_id': 5260607, 'url': 'https://thefed...","{'article_id': 2685025, 'url': 'https://www.na...","{'article_id': 534726, 'url': 'https://dailyca...","{'article_id': 4564071, 'url': 'https://www.da...","{'article_id': 1273620, 'url': 'https://www.da...","{'article_id': 1134021, 'url': 'https://dailyc...","{'article_id': 5919097, 'url': 'https://www.na...","{'article_id': 3443358, 'url': 'https://www.br...","{'article_id': 5932197, 'url': 'https://www.da...","{'article_id': 4092559, 'url': 'https://www.br..."


In [20]:
print_schema(temp.iloc[0,1])

|- article_id	 - <class 'int'>
|- url	 - <class 'str'>
|- title	 - <class 'str'>
|- text	 - <class 'str'>
|- source	 - <class 'str'>
|- source_partisan_score	 - <class 'float'>
|- tweet_id	 - <class 'int'>
|- tweet_screen_name	 - <class 'str'>
|- tweet_created_at	 - <class 'str'>
|- tweet_text	 - <class 'str'>
|- kws_label	 - list [3] <class 'str'>
|- cls_label	 - list [2] <class 'str'>
|- feature_vector	 - list [3000] <class 'float'>
|- topical_vector	 - list [70] <class 'int'>


In [21]:
def make_filter(temp, dict_keys):
  filter = pd.DataFrame(columns=dict_keys)

  for i in range(temp.shape[1]):
    row = temp.iloc[0, i]
    new = [row[key] for key in dict_keys]

    filter.loc[i] = new

  return filter

In [22]:
bert_filter = make_filter(temp, ['article_id', 'title', 'text', 'source_partisan_score'])

In [23]:
bert_filter.to_csv('bert_filter.csv')

In [24]:
### train / test / val split - masks generated

In [25]:
perm = make_filter(temp, ['article_id', 'source_partisan_score', 'cls_label'])

In [26]:
topic_association = perm.copy()

In [27]:
import numpy as np

In [28]:
for row_id in range(topic_association.shape[0]):
  topics = np.array(topic_association.iloc[row_id]['cls_label'])
  r = np.random.randint(0, topics.size)

  topic_association.iloc[row_id, topic_association.columns.get_loc('cls_label')] = topics[r]




In [29]:
topic_association.head()

,article_id,source_partisan_score,cls_label
0,1741577,-2.0,health care
1,2141595,-2.0,trump impeachment
2,6556505,-2.0,racism
3,2867824,-2.0,environment
4,5703814,-2.0,us military


In [30]:
topic_association.to_csv('even_filter_base.csv')

In [31]:
filter_base = pd.read_csv('even_filter_base.csv')

In [32]:
df = filter_base
df['stratify_key'] = df.apply(lambda row: (row['source_partisan_score'], row['cls_label']), axis=1)

In [33]:
df.head()

,Unnamed: 0,article_id,source_partisan_score,cls_label,stratify_key
0,0,1741577,-2.0,health care,"(-2.0, health care)"
1,1,2141595,-2.0,trump impeachment,"(-2.0, trump impeachment)"
2,2,6556505,-2.0,racism,"(-2.0, racism)"
3,3,2867824,-2.0,environment,"(-2.0, environment)"
4,4,5703814,-2.0,us military,"(-2.0, us military)"


In [34]:
from sklearn.model_selection import train_test_split

# First split: train+val (90%) and test (10%)
train_val, test = train_test_split(
    df, test_size=0.1, stratify=df['stratify_key'], random_state=42
)

# Second split: train (80%) and val (10%)
train, val = train_test_split(
    train_val, test_size=0.1111, stratify=train_val['stratify_key'], random_state=42
)

In [35]:
train.to_csv('training_mask.csv')
test.to_csv('testing_mask.csv')
val.to_csv('validation_mask.csv')

In [36]:
print(train.shape)
print(test.shape)
print(val.shape)

(32000, 5)
(4000, 5)
(4000, 5)


In [37]:
bert_set = pd.read_csv('bert_filter.csv')

In [38]:
def filter_by_id(s, filtering_table, filter_col):
    filter = filtering_table[filter_col]
    new_set = s[s[filter_col].isin(filter)]
    ##print(new_set.shape)
    new_set.drop(columns=['Unnamed: 0'], inplace=True)
    ##print(new_set.head)
    return new_set

In [39]:
train_set = filter_by_id(bert_set, train, 'article_id')
test_set = filter_by_id(bert_set, test, 'article_id')
val_set = filter_by_id(bert_set, val, 'article_id')

<ipython-input-38-8868b9285add>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_set.drop(columns=['Unnamed: 0'], inplace=True)
<ipython-input-38-8868b9285add>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_set.drop(columns=['Unnamed: 0'], inplace=True)
<ipython-input-38-8868b9285add>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_set.drop(columns=['Unnamed: 0'], inplace=True)


In [40]:
train_set.head()

,article_id,title,text,source_partisan_score
1,2141595,Republicans made absurd defenses of Trump duri...,On the first day of public impeachment inquiry...,-2.0
2,6556505,ER doctor: We must do better on COVID-19 messa...,As the world grapples with the devastation of ...,-2.0
4,5703814,Venezuela Coup Fiasco Doesn’t Hide U.S. Goal o...,"Two days later, the Venezuelan foreign ministe...",-2.0
5,1267386,Punishing the poor for being hungry,The United States might be the only country in...,-2.0
6,5140948,"Trump announces halt to WHO funding, NYC revis...",President Donald Trump has announced he will s...,-2.0


In [41]:
train_set = train_set.sample(frac=1)
train_set = train_set.sample(frac=1)
train_set = train_set.sample(frac=1)
train_set = train_set.sample(frac=1)

In [42]:
test_set = test_set.sample(frac=1)
test_set = test_set.sample(frac=1)
test_set = test_set.sample(frac=1)
test_set = test_set.sample(frac=1)

In [43]:
val_set = val_set.sample(frac=1)
val_set = val_set.sample(frac=1)
val_set = val_set.sample(frac=1)
val_set = val_set.sample(frac=1)

In [44]:
train_set.to_csv('bert_training_set.csv')
test_set.to_csv('bert_testing_set.csv')
val_set.to_csv('bert_validation_set.csv')

In [45]:
######### Dataset to generate bert embeddings over

In [46]:
# bert_filter = make_filter(temp, ['article_id', 'title', 'text'])

In [47]:
# bert_filter.head()

In [48]:
# bert_filter.describe()

In [49]:
# bert_filter.to_csv('bert_filter.csv')